In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_openai import AzureChatOpenAI

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv()


In [4]:
LLMclient = AzureChatOpenAI(
    azure_deployment=os.getenv('MODEL_NAME'),
    azure_endpoint=os.getenv('BASE_API'),
    api_key=os.getenv('API_KEY'),
    api_version=os.getenv('API_VERSION'),  # Ensure the correct API version is used
    model=os.getenv('MODEL_NAME'),
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [8]:
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone"],
    template= TEMPLATE
)

In [14]:
quiz_chain1 = LLMChain(llm=LLMclient,prompt=quiz_prompt,output_key="quiz",verbose= True)

In [16]:
quiz_chain1

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001CA169ED7D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001CA16796FD0>, model_name='GPT4-Turbo', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://deployment-gpt4.openai.azure.com', deployment_name='GPT4-Turbo', openai_api_version='2024-02-15-preview', open

In [19]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [20]:
quiz_prompt2 = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [21]:
quiz_chain2 = LLMChain( llm=LLMclient,prompt=quiz_prompt2,output_key="resultKey",verbose= True)

In [22]:
connect_chains = SequentialChain(chains=[quiz_chain1,quiz_chain2],output_variables=["quiz","resultKey"],input_variables=["text", "number", "subject", "tone", "response_json"], verbose=True)

In [23]:
file_path=r"C:\Users\MQ829CF\Documents\LearningSpace\TryMCQ\data.txt"
with open(file_path,encoding="utf8",mode="r") as file:
    TEXT = file.read()

In [24]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER=5 
SUBJECT="Biodiversity"
TONE="simple"

In [27]:
with get_openai_callback() as cb:
    response=connect_chains(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\MQ829CF\Documents\LearningSpace\TryMCQ\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:According to the Oxford English Dictionary (third edition 2009), the name "India" is derived from the Classical Latin India, a reference to South Asia and an uncertain region to its east. In turn the name "India" derived successively from Hellenistic Greek India ( Ἰνδία), ancient Greek Indos ( Ἰνδός), Old Persian Hindush (an eastern province of the Achaemenid Empire), and ultimately its cognate, the Sanskrit Sindhu, or "river", specifically the Indus River and, by implication, its well-settled southern basin.[73][74] The ancient Greeks referred to the Indians as Indoi (Ἰνδοί), which translates as "The people of the Indus".[75]

The term Bharat (Bhārat; pronounced [ˈbʱaːɾət] ⓘ), mentioned in both Indian epic poetry and the Constitution of India,[76][77] is used in its variations by many Indian languages. A modern rendering of the historical name Bharatavarsha, which applied origin

In [36]:
response

{'text': 'According to the Oxford English Dictionary (third edition 2009), the name "India" is derived from the Classical Latin India, a reference to South Asia and an uncertain region to its east. In turn the name "India" derived successively from Hellenistic Greek India ( Ἰνδία), ancient Greek Indos ( Ἰνδός), Old Persian Hindush (an eastern province of the Achaemenid Empire), and ultimately its cognate, the Sanskrit Sindhu, or "river", specifically the Indus River and, by implication, its well-settled southern basin.[73][74] The ancient Greeks referred to the Indians as Indoi (Ἰνδοί), which translates as "The people of the Indus".[75]\n\nThe term Bharat (Bhārat; pronounced [ˈbʱaːɾət] ⓘ), mentioned in both Indian epic poetry and the Constitution of India,[76][77] is used in its variations by many Indian languages. A modern rendering of the historical name Bharatavarsha, which applied originally to North India,[78][79] Bharat gained increased currency from the mid-19th century as a nat

In [29]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:7277
Prompt Tokens:6468
Completion Tokens:809
Total Cost:0.24258


In [37]:
quiz_data=response.get("quiz")


In [38]:
quiz_data

'```json\n{\n  "1": {\n    "mcq": "What percentage of India\'s total land area is covered by forest?",\n    "options": {\n      "a": "15.71%",\n      "b": "21.71%",\n      "c": "25.32%",\n      "d": "30.05%"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Which of the following species is considered vulnerable in India?",\n    "options": {\n      "a": "Bengal tiger",\n      "b": "Ganges river dolphin",\n      "c": "Hooded leaf monkey",\n      "d": "Indian white-rumped vulture"\n    },\n    "correct": "c"\n  },\n  "3": {\n    "mcq": "How many biodiversity hotspots are present in India?",\n    "options": {\n      "a": "2",\n      "b": "4",\n      "c": "6",\n      "d": "8"\n    },\n    "correct": "b"\n  },\n  "4": {\n    "mcq": "What is the percentage of mammal species that India is a habitat for?",\n    "options": {\n      "a": "8.6%",\n      "b": "12.6%",\n      "c": "13.7%",\n      "d": "7.9%"\n    },\n    "correct": "a"\n  },\n  "5": {\n    "mcq": "Which Indian tree is widely

In [42]:
quiz_data=json.loads(response.get("quiz"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)